# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import googlemaps

# Import API key
from api_keys import g_key

### Use csv from Weather Py 

In [2]:
weather_file = pd.read_csv("../output_data/city.csv")
weather_file.dropna()
weather_file


,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Verkh-Chebula,91,RU,1619912616,85,56.0308,87.6218,37.45,10.76
1,1,San Quintín,95,MX,1619912616,71,30.4833,-115.9500,64.62,17.00
2,2,Upernavik,8,GL,1619912617,83,72.7868,-56.1549,23.31,5.79
3,3,Valley,90,US,1619912617,81,45.1277,-116.1262,55.00,5.75
4,4,Kapaa,20,US,1619912617,69,22.0752,-159.3190,80.60,16.11
...,...,...,...,...,...,...,...,...,...,...
62,62,Norman Wells,20,CA,1619912632,34,65.2820,-126.8329,48.20,13.80
63,63,Nemuro,75,JP,1619912633,75,43.3236,145.5750,41.00,11.50
64,64,Arraial do Cabo,100,BR,1619912633,68,-22.9661,-42.0278,72.99,12.44
65,65,Mweka,100,CD,1619912633,99,-4.8500,21.5667,67.96,2.04


### Humidity Heatmap


In [3]:
#Load gmaps gkey
gmaps.configure(api_key=g_key)

In [4]:

locations = weather_file[["Lat", "Lng"]]
humidity = weather_file["Humidity"].astype(float)

In [5]:
hfigure = gmaps.figure(center = [0,0] ,zoom_level = 2)

#Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
hfigure.add_layer(heat_layer)
#Display figure
hfigure

Figure(layout=FigureLayout(height='420px'))

### Cities meeting ideal weather conditions

In [6]:
temp_file=weather_file.loc[(weather_file['Max Temp']>=60)&(weather_file['Max Temp']<=90)]
wind_file=temp_file.loc[(temp_file['Wind Speed']<=10)]
ideal_weather=wind_file.loc[(wind_file['Cloudiness']<=10)]
ideal_weather



,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
12,12,Cabo San Lucas,1,MX,1619912619,47,22.8909,-109.9124,79.00,6.91
24,24,Port Alfred,0,ZA,1619912539,65,-33.5906,26.8910,60.73,5.79
27,27,Atuona,2,PF,1619912238,68,-9.8000,-139.0333,79.39,6.82


### Hotel Map

In [7]:
possible_hotels = ideal_weather.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
possible_hotels["Hotel Name"] = ""
possible_hotels

,City,Country,Lat,Lng,Hotel Name
12,Cabo San Lucas,MX,22.8909,-109.9124,
24,Port Alfred,ZA,-33.5906,26.8910,
27,Atuona,PF,-9.8000,-139.0333,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [ ]:
for index, row in possible_hotels.iterrows():
    
    # get lat, lng 
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    #base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    json.dumps(hotel_name, indent=4, sort_keys=True)
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        possible_hotels.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in possible_hotels.iterrows()]
locations = possible_hotels[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

hfigure.add_layer(markers)

# Display Map
hfigure
